# START

In [23]:
import sys
sys.path.append(r'E:\python-test\study-python')

## 数据集

In [24]:
from datasets import load_dataset
loaded_dataset = load_dataset(r"E:\python-test\DATASETS\zxbsmkwebnovel_cn")

In [25]:
loaded_dataset['train'][0]

{'input': '有其主必有其仆，咱们张大少爷的话还没有说完，张石头的口水就已经流出了嘴角，张大少爷又趁热打铁，鼓动道：“你想想，熊瑚那个丫鬟秀儿长得有多漂亮啊？你如果一辈子窝在这个临清城里，有没有可能讨到这么漂亮的媳妇？这么好的机会放在面前，难道你想错过，以后就只抱着一个黄脸婆过下半辈子？”\n张大少爷的话句句都打中要害，二十出头还没讨到老婆的张石头难免动心无比，不过仔细考虑了一下后，张石头还是哭丧着脸说道：“少爷，不是小的拦着你，只是这事可不是开玩笑，你从小就没离开过临清城，忽然之间悄悄离开家进京，老爷和十一位夫人怎么可能放心得下？只怕你前脚出了城，老爷后脚就派人把你抓回家了。而且还有一点，那只母老虎熊瑚实在太凶了，我们进了京城就算能找得到她，只怕少爷你也没办法把她娶回家啊？”\n“怕什么？只要功夫深，铁棒磨成针，咱们只要再见到熊瑚那只母老虎，就肯定有办法把她连人带丫鬟一起弄回临清。”早有打算却不能明言的张大少爷随口瞎扯，又皱眉道：“至于老头子这边，是得防着他派人来抓我们，否则他回来发现少爷我不在家，肯定要到处去找。”琢磨到这里，张大少爷一拍大腿，喜道：“对了，有办法让老头子抓不到咱们了。”\n“什么办法？”张石头赶紧问道。张大少爷奸笑道：“别急，到时候你就知道了。别罗嗦了，快回家收拾行李，乘着还没天黑赶快动身。”',
 'instruction': '给你一段话，帮我继续写下去',
 'output': '张石头将信将疑，又实在细胳臂扭不过大腿，只得咬牙决定赌上一把，“好吧，既然少爷你执意要去，小的就陪你走上这一趟。不过将来老爷如果追究起来，少爷你可要给小的说几句好话。”张大少爷咧嘴一笑，安慰道：“放心吧，如果老头子真的把我们抓住了，少爷我全抗下来，不关你事。”\n……\n好不容易说服张石头同行，当下张大少爷和张石头一主一仆两个活宝立即行动起来，先是翻墙回家分头行动，张石头摸到张大少爷房间里收拾衣服行李，张大少爷则通过记忆找出前任张大少爷偷配的银库钥匙摸进张老财的书房，打开张老财用来收藏贵重财物的密库，搜刮出厚厚一大叠银票和满满一大包金叶子金元宝，又装了不少极其贵重的珠宝，这才偷偷摸回自己的房间与张石头会合。\n刚进门，张石头就举起一个包裹说岛：“少爷，东西都准备好了，按你的吩咐，你和我都只带了一套换洗的衣服，还有一点零用的碎银子。”张大少爷点点头，又让张

### TOKENIZER

In [26]:
from transformers import AutoTokenizer
text_data = loaded_dataset['train'][0]['input']
# 创建 tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2", cache_dir=r'./TOKENIZER')  # 可以根据需要选择基础模型


In [42]:
# 加载数据集
dataset = loaded_dataset

# 应用tokenizer到数据集
def apply_tokenizer(example):
    return tokenizer(example['input'], example['instruction'], example['output'], truncation=True,max_length=512)

tokenized_dataset = dataset.map(apply_tokenizer, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [47]:
tokenized_dataset['train'][0].keys(),tokenized_dataset['train'][0]

(dict_keys(['input_ids', 'attention_mask', 'labels']),
 {'input_ids': tensor([17312,   231, 17739,   114, 10310,   119, 33232,   227, 17312,   231,
          17739,   114, 20015,   228,   171,   120,   234,   161,   240,   109,
          20015,   105, 28156,   254, 32014, 22887,   239,   163,   230,   115,
          21410, 46237,   251, 32573,   246,   162,   110,    94, 17312,   231,
          46237,   112, 22522,   234,   171,   120,   234, 28156,   254,   163,
            253,   111, 13783,   112, 21410, 20998,    96, 36365,   112, 22887,
            109, 32432,   110,   163,   119,   237, 38184,   223, 49035,   118,
          12859,   228,   161,   246,   112,   164,   100,   240,   171,   120,
            234, 28156,   254, 32014, 22887,   239,   163,   230,   115, 20998,
          42062,   114,   223,   163,   225,   255, 33699,   241,   165,   241,
            223,   171,   120,   234,   165,   120,   241, 27950,   101, 34402,
            241,   171,   120,   248,   447,   250, 

In [45]:
tokenized_dataset.set_format("torch", columns=["input_ids",'attention_mask', "labels"])

In [46]:
tokenized_dataset['train'][600]['labels'].shape

torch.Size([512])

## infini-llama

In [ ]:
%load_ext autoreload
%autoreload 2

from tot.infini_llama import *
from transformers import LlamaConfig

In [ ]:
from transformers import LlamaConfig, LlamaForCausalLM, Trainer, TrainingArguments

config = LlamaConfig()
config.hidden_size=256
config.intermediate_size=172
config.max_position_embeddings=256
config.num_attention_heads=1
config.num_hidden_layers=1
config.num_key_value_heads=1
config.rope_theta=10.0
config.vocab_size=200

# 配置模型
model = LlamaForCausalLM(config=config)


In [ ]:
tokenized_dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader, IterableDataset
import torch

# 创建DataLoader对象
train_loader = DataLoader(tokenized_dataset['train'], batch_size=32, shuffle=True)

next(iter(train_loader))


In [ ]:

# 训练循环示例
for epoch in range(num_epochs):
    for batch in train_loader:
        # 假设batch是一个包含多个样本的元组
        input_ids, attention_mask, labels = batch
        
        # 这里添加模型的前向传播、计算损失、反向传播和优化器步骤
        pass

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

# 创建数据集和数据加载器
dataset = TextDataset(texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=2)

# 加载预训练的BERT模型
model = EncoderDecoderModel.from_pretrained('bert-base-uncased')

# 定义损失函数和优化器
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# 训练模型
model.train()
for epoch in range(3):  # 简单的示例，只训练3个epoch
    for batch in dataloader:
        inputs = batch['input_ids']
        outputs = model(input_ids=inputs, decoder_input_ids=inputs[:, :-1])
        loss = loss_fn(outputs.view(-1, outputs.shape[-1]), inputs[:, 1:].view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

print("Training complete.")